In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [2]:
# For reproducibility
torch.manual_seed(1)

In [3]:
# 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [4]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079378


In [5]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [6]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [7]:
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

## High-level Implementation with nn.module

In [20]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [21]:
import numpy as np

import torch.nn as nn

# nn모듈을 사용
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1) # 입력 3, 출력 1

    def forward(self, x):         # hypothesis 계산 : forward()에서, backward는 알아서 해줌 
        return self.linear(x)

model = MultivariateLinearRegressionModel()

In [28]:
# optimizer, # (model.parameters(), lr)
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [29]:
x_train

tensor([[ 73.,  80.,  75.],
        [ 93.,  88.,  93.],
        [ 89.,  91.,  90.],
        [ 96.,  98., 100.],
        [ 73.,  66.,  70.]])

In [30]:
# cost 계산
nb_epochs = 200
for epoch in range(nb_epochs + 1) :
    # H(x) 계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(hypothesis, y_train)
    
    # cost로 H(x)개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis : {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/200 hypothesis : tensor([-20.5930, -30.8318, -27.2626, -29.8314, -24.8466]) Cost: 39633.414062
Epoch    1/200 hypothesis : tensor([57.1418, 62.6022, 64.7982, 70.4206, 46.4205]) Cost: 12431.273438
Epoch    2/200 hypothesis : tensor([100.6621, 114.9128, 116.3395, 126.5479,  86.3209]) Cost: 3904.840332
Epoch    3/200 hypothesis : tensor([125.0270, 144.2000, 145.1954, 157.9714, 108.6602]) Cost: 1232.251709
Epoch    4/200 hypothesis : tensor([138.6674, 160.5973, 161.3506, 175.5641, 121.1676]) Cost: 394.532684
Epoch    5/200 hypothesis : tensor([146.3035, 169.7779, 170.3951, 185.4135, 128.1707]) Cost: 131.948273
Epoch    6/200 hypothesis : tensor([150.5782, 174.9182, 175.4586, 190.9277, 132.0919]) Cost: 49.637566
Epoch    7/200 hypothesis : tensor([152.9708, 177.7964, 178.2933, 194.0147, 134.2878]) Cost: 23.833128
Epoch    8/200 hypothesis : tensor([154.3098, 179.4082, 179.8802, 195.7429, 135.5177]) Cost: 15.740547
Epoch    9/200 hypothesis : tensor([155.0588, 180.3111, 180.7684, 

## loading Data

* 대부분 데이터 셋은 적어도 수십만 개의 데이터를 제공
* 너무 많은 데이터를 한번에 학습시키기에는 너무 느리고 하드웨어적으로 불가능 -> 균일하게 나눠서 학습 : minibatch gradient descent

### mini-batch gradient descent 
* 업데이트를 좀 더 빠르게 할 수 있다.
* 전체 데이터를 쓰지 않아서 잘못된 방향으로 업데이트를 할 수도 있다. 


### PyTorch Dataset

In [32]:
# pytorch dataset 생성
from torch.utils.data import Dataset

class CustomDataset(Dataset) :
    def __init__(self) : 
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
    
    # 길이를 반환하는 매직 method 설정
    def __len__(self) :
        return len(self.x_data)
    
    # 인덱스를 받았을 때 그에 맞는 데이터를 반환하는 method
    def __getitem__(self, idx) :
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y
    
dataset = CustomDataset()

In [35]:
# dataset을 만들면 dataloader를 사용할 수 있음
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset, 
    batch_size=2,           # minibatch의 크기
    shuffle=True,           # Epoch마다 데이터셋을 섞어서, 데이터가 학습되는 순서를 바꿈 
)


In [36]:
epochs = 20
for epoch in range(epochs + 1) :
    for batch_idx, samples in enumerate(dataloader) :
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        

        print('Epoch {:4d}/{} Batch : {}/{} Cost: {:.6f}'.format(
            epoch, epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/20 Batch : 1/3 Cost: 16.692860
Epoch    0/20 Batch : 2/3 Cost: 13.284204
Epoch    0/20 Batch : 3/3 Cost: 13.311319
Epoch    1/20 Batch : 1/3 Cost: 2.854760
Epoch    1/20 Batch : 2/3 Cost: 20.381283
Epoch    1/20 Batch : 3/3 Cost: 17.847067
Epoch    2/20 Batch : 1/3 Cost: 12.155276
Epoch    2/20 Batch : 2/3 Cost: 13.486388
Epoch    2/20 Batch : 3/3 Cost: 6.275967
Epoch    3/20 Batch : 1/3 Cost: 6.307155
Epoch    3/20 Batch : 2/3 Cost: 29.522331
Epoch    3/20 Batch : 3/3 Cost: 6.708002
Epoch    4/20 Batch : 1/3 Cost: 1.793839
Epoch    4/20 Batch : 2/3 Cost: 25.916840
Epoch    4/20 Batch : 3/3 Cost: 21.124273
Epoch    5/20 Batch : 1/3 Cost: 9.469073
Epoch    5/20 Batch : 2/3 Cost: 12.709829
Epoch    5/20 Batch : 3/3 Cost: 16.598015
Epoch    6/20 Batch : 1/3 Cost: 26.860384
Epoch    6/20 Batch : 2/3 Cost: 7.953699
Epoch    6/20 Batch : 3/3 Cost: 10.997133
Epoch    7/20 Batch : 1/3 Cost: 13.737206
Epoch    7/20 Batch : 2/3 Cost: 13.789694
Epoch    7/20 Batch : 3/3 Cost: 14.364049